In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import boto3, json
from constants import ModelIDs, Temperature

session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

message_list = []

model_id = ModelIDs.anthropic_claude_3_sonnet
temp = Temperature.FOCUSED

In [8]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "entities": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "name": {"type": "string", "description": "The extracted entity name."},
                    "type": {"type": "string", "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION)."},
                    "context": {"type": "string", "description": "The context in which the entity appears in the text."}
                  },
                  "required": ["name", "type", "context"]
                }
              }
            },
            "required": ["entities"]
          }
        }
      }
    }
  ]
}

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "messages": [{"role": "user", "content": [{"text": query}]}],
    "additionalModelRequestFields": {"max_tokens": 4096},
    "toolConfig": toolConfig
}

response = bedrock.converse(**converse_api_params)


json_entities = None
for content in response['output']['message']['content']:
    if isinstance(content, dict) and 'toolUse' in content:
        tool_use = content['toolUse']
        if tool_use['name'] == "print_entities":
            json_entities = tool_use['input']
            break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{
  "entities": [
    {
      "name": "John",
      "type": "PERSON",
      "context": "John works at Google in New York."
    },
    {
      "name": "Google",
      "type": "ORGANIZATION",
      "context": "John works at Google in New York."
    },
    {
      "name": "New York",
      "type": "LOCATION",
      "context": "John works at Google in New York."
    },
    {
      "name": "Sarah",
      "type": "PERSON",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    },
    {
      "name": "Acme Inc.",
      "type": "ORGANIZATION",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    },
    {
      "name": "San Francisco",
      "type": "LOCATION",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    }
  ]
}


In [3]:
initial_message = {
    "role": "user",
    "content": [
        { "text": "How are you today?" } 
    ],
}

message_list.append(initial_message)

response = bedrock.converse(
    modelId= model_id,
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": temp
    },
)

response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

{
    "role": "assistant",
    "content": [
        {
            "text": "I'm doing well, thanks for asking! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest."
        }
    ]
}


In [4]:
message_list.append(response_message)

print(json.dumps(message_list, indent=4))

[
    {
        "role": "user",
        "content": [
            {
                "text": "How are you today?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'm doing well, thanks for asking! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest."
            }
        ]
    }
]


In [5]:
with open("image.webp", "rb") as image_file:
    image_bytes = image_file.read()

image_message = {
    "role": "user",
    "content": [
        { "text": "Image 1:" },
        {
            "image": {
                "format": "webp",
                "source": {
                    "bytes": image_bytes #no base64 encoding required!
                }
            }
        },
        { "text": "Please describe the image." }
    ],
}

message_list.append(image_message)

response = bedrock.converse(
    modelId=model_id,
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": temp
    },
)

response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

message_list.append(response_message)


{
    "role": "assistant",
    "content": [
        {
            "text": "The image shows a miniature model of a house, likely a decorative or toy item. The house has a blue exterior with white window frames and a red tiled roof. It appears to be made of a ceramic or porcelain material. The miniature house is placed on a surface with some greenery and yellow flowers surrounding it, creating a whimsical and natural setting. The background is slightly blurred, drawing focus to the intricate details of the small house model."
        }
    ]
}


In [6]:
image_message = {
    "role": "user",
    "content": [
        { "text": "Image 1:" },
        {
            "image": {
                "format": "webp",
                "source": {
                    "bytes": image_bytes #no base64 encoding required!
                }
            }
        },
        { "text": "Write a potery describing the image" }
    ],
}

message_list.append(image_message)

response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 1
    },
)

response_message = response['output']['message']
print(json.dumps(response_message, indent=4))

message_list.append(response_message)

{
    "role": "assistant",
    "content": [
        {
            "text": "Here's a short poem describing the charming miniature house in the image:\n\nTiny Dwelling, Whimsical Sight\nA miniature abode, a delight\nBlue walls and red-tiled roof so bright\nWindows aglow with warm candlelight\n\nNestled midst greenery, flowers golden\nA scene from storybooks untolden\nInviting dreams, adventures beholden\nIn this sweet world, so small and embolden\n\nA whimsical treasure, a joy to behold\nThis little house's charms unfold\nBringing smiles to young and old\nA precious memory to forever hold"
        }
    ]
}


In [34]:
tool_list = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function."
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    }
]

message_list = []

initial_message = {
    "role": "user",
    "content": [
        { "text": "What is the cosine of 7?" } 
    ],
}

system_message = [
    {"text":"You must only do math by using a tool."}
]

message_list.append(initial_message)

response = bedrock.converse(
    modelId=model_id,
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": temp
    },
    toolConfig={
        "tools": tool_list
    },
    system=system_message
)

In [35]:
response_message = response['output']['message']
print(json.dumps(response_message, indent=4))
message_list.append(response_message)

{
    "role": "assistant",
    "content": [
        {
            "text": "Here is how we can calculate the cosine of 7 using the available tool:"
        },
        {
            "toolUse": {
                "toolUseId": "tooluse_SCoGD_alR3Wf1B-MfZNhJg",
                "name": "cosine",
                "input": {
                    "x": 7
                }
            }
        }
    ]
}


In [36]:
import boto3, json, math

response_content_blocks = response_message['content']

for content_block in response_content_blocks:
    if 'toolUse' in content_block:
        tool_use_block = content_block['toolUse']
        tool_use_name = tool_use_block['name']
        
        print(f"Using tool {tool_use_name}")
        
        if tool_use_name == 'cosine':
            tool_result_value = math.cos(tool_use_block['input']['x'])
            print(tool_result_value)
            
    elif 'text' in content_block:
        print(content_block['text'])


Here is how we can calculate the cosine of 7 using the available tool:
Using tool cosine
0.7539022543433046


In [37]:
follow_up_content_blocks = []

for content_block in response_content_blocks:
    if 'toolUse' in content_block:
        tool_use_block = content_block['toolUse']
        tool_use_name = tool_use_block['name']
        
        print(f"Using tool {tool_use_name}")
        if tool_use_name == 'cosine':
            tool_result_value = math.cos(tool_use_block['input']['x'])
            print(tool_result_value)
            follow_up_content_blocks.append({
                "toolResult": {
                    "toolUseId": tool_use_block['toolUseId'],
                    "content": [
                        {
                            "json": {
                                "result": tool_result_value
                            }
                        }
                    ]
                }
            })
    elif 'text' in content_block:
        print(content_block['text'])

Here is how we can calculate the cosine of 7 using the available tool:
Using tool cosine
0.7539022543433046


In [38]:
if len(follow_up_content_blocks) > 0:
    
    follow_up_message = {
        "role": "user",
        "content": follow_up_content_blocks,
    }
    
    message_list.append(follow_up_message)
    print(json.dumps(message_list, indent=4))
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0
        },
        toolConfig={
            "tools": tool_list
        },
        system=[{"text":"You must only do math by using a tool."}]
    )
    
    response_message = response['output']['message']
    print(json.dumps(response_message, indent=4))
    message_list.append(response_message)


[
    {
        "role": "user",
        "content": [
            {
                "text": "What is the cosine of 7?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Here is how we can calculate the cosine of 7 using the available tool:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_SCoGD_alR3Wf1B-MfZNhJg",
                    "name": "cosine",
                    "input": {
                        "x": 7
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_SCoGD_alR3Wf1B-MfZNhJg",
                    "content": [
                        {
                            "json": {
                                "result": 0.7539022543433046
                            }
                        }
          

In [39]:
print(json.dumps(message_list, indent=4))

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is the cosine of 7?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Here is how we can calculate the cosine of 7 using the available tool:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_SCoGD_alR3Wf1B-MfZNhJg",
                    "name": "cosine",
                    "input": {
                        "x": 7
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_SCoGD_alR3Wf1B-MfZNhJg",
                    "content": [
                        {
                            "json": {
                                "result": 0.7539022543433046
                            }
                        }
          